In [96]:
# Imports
import pandas as pd
import numpy as np

In [66]:
# Getting Labels
with open('data/MANIFEST.txt') as read:
    labels = list(read)
    for i in range(0, len(labels)):
        labels[i] = labels[i].rstrip('\n') # Not sure how else to read this in... there may be a better way

In [99]:
# Test for format
# Add to file list


files = dict() # Dictionary - key = label, value = list of filenames
features = list()
bad_files = list()

first_manifest = pd.read_csv('data/' + labels[0] + '/MANIFEST.txt', delimiter='\t')
first_file = pd.read_csv('data/' + labels[0] + '/' + first_manifest.at[0, 'filename'], delimiter='\t')
feature_order = first_file['miRNA_ID']
features = feature_order

for label in labels:
    print('starting with label: ', label)
    
    manifest = pd.read_csv('data/' + label + '/MANIFEST.txt', delimiter='\t')
    filenames = manifest['filename']
    
    files[label] = []
    
    for i in range(0, len(filenames)):
        filename = filenames[i]
        
        if filename.endswith('annotations.txt'):
            filenames.drop(i)
        else:
            file = pd.read_csv('data/' + label + '/' + filename, delimiter='\t')
            file_features = file['miRNA_ID']
            
            if not file_features.equals(feature_order):
                bad_files.append(filename)
            else:
                files[label].append(filename)
            
        
if len(bad_files) != 0:
    print(bad_files)
else:
    print('no bad files! :)')

starting with label:  Breast Invasive Carcinoma
starting with label:  Kidney Renal Clear Cell Carcinoma
starting with label:  Lung Adenocarcinoma
starting with label:  Lung Squamous Cell Carcinoma
starting with label:  Pancreatic Adenocarcinoma
starting with label:  Uveal Melanoma
no bad files! :)


In [101]:
# Reading in files

values_list = list()

for label in labels:
    for file in files[label]:
        file_frame = pd.read_csv('data/' + label + '/' + file, delimiter='\t')
        values_list.append(file_frame['miRNA_ID'])
        
data = pd.DataFrame(values_list, columns=features)
data.head()

miRNA_ID,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
miRNA_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
miRNA_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
miRNA_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
miRNA_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
miRNA_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
